<a href="https://colab.research.google.com/github/AbhirKarande/OCRandProductRecognition/blob/main/RFClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install python3.9


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9 python3.9-minimal
0 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 5,030 kB of archives.
After this operation, 19.7 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.9-minimal amd64 3.9.17-1+focal1 [805 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.9-minimal amd64 3.9.17-1+focal1 [2,064 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.9-stdlib amd64 3.9.17-1+focal1 [1,666 kB]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.9 amd64 3.

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

update-alternatives: using /usr/bin/python3.9 to provide /usr/bin/python3 (python3) in auto mode


In [ ]:
!python --version


Python 3.9.17


In [ ]:
!git clone https://github.com/AbhirKarande/OCRandProductRecognition.git

Cloning into 'OCRandProductRecognition'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 176 (delta 4), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (176/176), 50.54 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
import shutil

# Specify the path of the directory you want to download
directory_path = "/content/OCRandProductRecognition/WholeFoodsTrainingImages/train"

# Create a zip file of the directory
shutil.make_archive("/content/whole_foods_training_images", "zip", directory_path)

# Download the zip file
from google.colab import files
files.download("/content/whole_foods_training_images.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!mkdir basil

mkdir: cannot create directory ‘basil’: File exists


In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=2057bc65990b05ff9d13f84812adaa1a52fe4cb952c56b744b75ad82671e6234
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn


In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 1.2.2.


In [ ]:
# Define the path to your training images
training_path = "/content/OCRandProductRecognition/whole_foods_training_images"

# Initialize empty lists for images and labels
images = []
labels = []

In [ ]:
print(labels)

[]


In [ ]:
for subdir in os.listdir(training_path):
    subdir_path = os.path.join(training_path, subdir)
    if os.path.isdir(subdir_path):
        # Read and preprocess images within each subdirectory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if os.path.isfile(file_path):
                image = cv2.imread(file_path)
                print(file_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (100, 100))  # Resize images if needed
                images.append(image)
                labels.append(subdir)

/content/OCRandProductRecognition/whole_foods_training_images/s1/13.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/11.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/2.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/5.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/3.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/10.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/7.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/4.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/9.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/8.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/6.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/1.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s1/12.jpg
/content/OCRandProductRecognition/whole_foods_training_images/s3/13.jpg
/

In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [ ]:
# Flatten the image arrays
images = images.reshape(images.shape[0], -1)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)


In [ ]:
# Create and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=135)
rf_classifier.fit(images, labels)


RandomForestClassifier(n_estimators=135)

In [ ]:
# Predict the labels for the test set
y_pred = rf_classifier.predict(X_test)

In [ ]:
print(images)

[[247 247 247 ... 247 247 247]
 [155 151 156 ... 198 188 185]
 [160 162 166 ... 182 178 172]
 ...
 [190 189 184 ...  31  69  86]
 [177 182 192 ... 107  53 101]
 [ 53  57  37 ... 159 157 154]]


In [ ]:
print(y_pred)

['s2' 's3' 's3' 's1' 's3' 's1']


In [ ]:
# Calculate and print the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [ ]:
!python --version

Python 3.9.17


In [ ]:
print(y_test)

['s2' 's3' 's3' 's1' 's3' 's1']


In [ ]:
print(y_pred)

['s2' 's3' 's3' 's1' 's3' 's1']


In [ ]:
import joblib
model_filename = "rfmodelnew2.pkl"
joblib.dump(rf_classifier, model_filename)

NameError: ignored

In [ ]:
import pickle

model_filename = "rfmodelnew1.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(rf_classifier, file)


NameError: ignored

In [ ]:
import pickle

model_filename = "/content/rfmodelnew1.pkl"

with open(model_filename, "rb") as file:
    loaded_model = pickle.load(file)

protocol_version = loaded_model.__reduce__()[2]
print("Protocol Version:", protocol_version)


EOFError: ignored

In [ ]:
!python --version

Python 3.9.17


In [ ]:
import joblib
import pickle
model_filename = "/content/rfmodelnew2.pkl"

loaded_model = pickle.load(open(model_filename, "rb"))

UnpicklingError: ignored

In [ ]:
import pickle

model_filename = "/content/rfmodelnew2.pkl"
protocols = range(pickle.HIGHEST_PROTOCOL + 1)
loaded_model = None

for protocol in protocols:
    try:
        with open(model_filename, "rb") as file:
            loaded_model = pickle.load(file)
        break
    except pickle.UnpicklingError:
        continue
    except EOFError:
        break

if loaded_model is None:
    print("Unable to load the pickled file.")

# Use the loaded_model object as needed


Unable to load the pickled file.


In [ ]:
!wget https://bootstrap.pypa.io/get-pip.py


--2023-07-10 19:07:46--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2578580 (2.5M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.46M  --.-KB/s    in 0.03s   

2023-07-10 19:07:46 (77.7 MB/s) - ‘get-pip.py’ saved [2578580/2578580]



In [ ]:
!sudo python3.9 get-pip.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.9 MB/s eta 0:00:00


In [ ]:
!python -m pip install --upgrade pip

/usr/bin/python3: No module named pip


In [ ]:
!pip install chassisml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other req

In [ ]:
import json
import pickle
import chassisml


In [ ]:
env = {
        "name": "chassis",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.9",
        {
            "pip": [
                "cv2",
                "sklearn",
                "numpy",
                "chassisml"
            ]
        }
    ]

}

In [ ]:
labels = ["s1", "s2", "s3", "s4"]

In [ ]:
print(loaded_model)

RandomForestClassifier(n_estimators=135)


In [ ]:
import cv2
import numpy as np

def process(input_bytes):
  img = cv2.imdecode(np.frombuffer(input_bytes, np.uint8),-1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (100, 100))  # Resize if needed
  img = img.reshape(1, -1)
  preds = loaded_model.predict_proba(img)
  inference_result = {
        "classPredictions": [
            {"row": i+1, "class": labels[np.argmax(pred)], "score": np.max(pred)} for i, pred in enumerate(preds)
        ]
    }

  structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }

# Print the predicted class

  return structured_output

In [ ]:
env1 = {
    "name": "chassis-conda-env",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.9", # or whatever python version you are using
        {
            "pip": [
                "chassisml",
                "opencv-python-headless",
                "numpy",
                "scikit-learn==1.2.2"
            ]
        }
    ]
}

In [ ]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com/")

# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)
sample_filepath = '/eggs.jpg'
results = chassis_model.test(sample_filepath)
print(results)


b'{"data":{"result":{"classPredictions":[{"row":1,"class":"s3","score":0.6370370370370371}]},"explanation":null,"drift":null}}'


In [ ]:
sample_filepath = '/9.jpeg'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"classPredictions":[{"row":1,"class":"s1","score":0.37333333333333335}]},"explanation":null,"drift":null}}'


In [ ]:
docker_user = ""
docker_pass = ""
model_name = "Grocery Item Classification10"

response = chassis_model.publish(
    model_name=model_name,
    model_version="0.0.1",
    registry_user=docker_user,
    registry_pass=docker_pass,
    conda_env=env1
)

Starting build job... 

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Ok!


In [ ]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
if final_status['status']['succeeded'] == 1:
    print("Job Completed. ")
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))

Job Completed. 


In [ ]:
!pip install modzy-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 17.2 MB/s eta 0:00:00
  Using cached boto3-1.27.1-py3-none-any.whl (135 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chassisml 1.4.13 requires protobuf==3.19.4, but you have protobuf 4.21.12 which is incompatible.
google-cloud-bigquery-connection 1.12.0 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.0, but you h

In [ ]:
from modzy import ApiClient, error
BASE_URL="https://tenant18.bot.5goil.tmobile.com/"
API_KEY="wZr4nkIAtLFPNUPySOdC.DYqB3325zWPXIDxdrZIH"
client=ApiClient(base_url=BASE_URL, api_key=API_KEY)

ERROR:modzy.http:unable to make network request
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 403, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/pyt

ValueError: ignored

In [ ]:
!pip install pyheif

In [ ]:
import os
from PIL import Image
import pyheif


def convert_heic_to_jpg(heic_file, output_folder):
    # Load the HEIC image
    heif_file = pyheif.read(heic_file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )

    # Generate the output file path
    filename = os.path.splitext(os.path.basename(heic_file))[0]
    output_file = os.path.join(output_folder, f"{filename}.jpg")

    # Save the image as JPG
    image.save(output_file, "JPEG")


def batch_convert_heic_to_jpg(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of HEIC files in the input folder
    heic_files = [
        file for file in os.listdir(input_folder)
        if file.lower().endswith(".heic")
    ]

    # Convert each HEIC file to JPG
    for heic_file in heic_files:
        heic_path = os.path.join(input_folder, heic_file)
        convert_heic_to_jpg(heic_path, output_folder)


# Specify the input and output folders
input_folder = "/content"
output_folder = "/content/test_images"

# Convert HEIC files to JPG
batch_convert_heic_to_jpg(input_folder, output_folder)


In [ ]:
import cv2
import numpy as np
import joblib

# Load the saved model from the file
loaded_model = joblib.load(model_filename)

# Preprocess the test image
test_image_path = "/content/test_images/IMG_3864.jpg"
test_image = cv2.imread(test_image_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_image = cv2.resize(test_image, (100, 100))  # Resize if needed
test_image = test_image.reshape(1, -1)  # Flatten the image array

# Make predictions on the test image
predicted_class = loaded_model.predict(test_image)

# Print the predicted class
print("Predicted class:", predicted_class[0])

Predicted class: s3


In [ ]:
!pip install chassisml

In [ ]:
import chassisml

In [ ]:
def process(input_bytes):
  img = cv2.imdecode(np.frombuffer(input_bytes, np.uint8),-1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (100, 100))  # Resize if needed
  img = img.reshape(1, -1)
  predicted_class = loaded_model.predict(test_image)

# Print the predicted class
  return predicted_class[0]

In [ ]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com/")
chassis_model = chassis_client.create_model(process_fn=process)

dockerhub_user = ''
dockerhub_pass = ''

response = chassis_model.publish(
   model_name="Sample ML Model",
   model_version="0.0.1",
   registry_user=dockerhub_user,
   registry_pass=dockerhub_pass
)

job_id = response.get('job_id')
print(job_id)
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!
chassis-builder-job-323a9a49-2ee0-4c09-a852-8ee6f692e366


In [ ]:
print(final_status)

{'logs': '\x1bINFO\x1b[0000] Retrieving image manifest continuumio/miniconda3:4.12.0 \n\x1bINFO\x1b[0000] Retrieving image continuumio/miniconda3:4.12.0 from registry index.docker.io \n\x1bINFO\x1b[0000] Built cross stage deps: map[]                \n\x1bINFO\x1b[0000] Retrieving image manifest continuumio/miniconda3:4.12.0 \n\x1bINFO\x1b[0000] Returning cached image manifest              \n\x1bINFO\x1b[0000] Executing 0 build triggers                   \n\x1bINFO\x1b[0000] Unpacking rootfs as cmd RUN apt-get update && apt-get install -y build-essential cmake requires it. \n\x1bINFO\x1b[0006] Taking snapshot of full filesystem...        \n\x1bINFO\x1b[0010] ARG MODEL_DIR                                \n\x1bINFO\x1b[0010] ARG MODEL_NAME                               \n\x1bINFO\x1b[0010] ARG MODEL_CLASS                              \n\x1bINFO\x1b[0010] ARG INTERFACE                                \n\x1bINFO\x1b[0010] ARG MODZY_METADATA_PATH                      \n\x1bINFO\x1b[0010] LABE

In [ ]:
print(final_status)

{'logs': '\x1bINFO\x1b[0000] Retrieving image manifest continuumio/miniconda3:4.12.0 \n\x1bINFO\x1b[0000] Retrieving image continuumio/miniconda3:4.12.0 from registry index.docker.io \n\x1bINFO\x1b[0000] Built cross stage deps: map[]                \n\x1bINFO\x1b[0000] Retrieving image manifest continuumio/miniconda3:4.12.0 \n\x1bINFO\x1b[0000] Returning cached image manifest              \n\x1bINFO\x1b[0000] Executing 0 build triggers                   \n\x1bINFO\x1b[0000] Unpacking rootfs as cmd RUN apt-get update && apt-get install -y build-essential cmake requires it. \n\x1bINFO\x1b[0005] Taking snapshot of full filesystem...        \n\x1bINFO\x1b[0009] ARG MODEL_DIR                                \n\x1bINFO\x1b[0009] ARG MODEL_NAME                               \n\x1bINFO\x1b[0009] ARG MODEL_CLASS                              \n\x1bINFO\x1b[0009] ARG INTERFACE                                \n\x1bINFO\x1b[0009] ARG MODZY_METADATA_PATH                      \n\x1bINFO\x1b[0009] LABE

In [ ]:
!pip install modzy-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:

from modzy import ApiClient
from modzy._util import file_to_bytes
client = ApiClient(base_url="https://tenant18.bot.5goil.tmobile.com/api", api_key="wZr4nkIAtLFPNUPySOdC")
sources = {}
#Add any number of inputs
sources["my-input"] = {
    "input.txt": "Lorem ipsum dolor sit amet",
}
#Once you are ready, submit the job
job = client.jobs.submit_text("ed542963de", "1.0.1", sources)
print(f"job: {job}")



ERROR:modzy.http:unable to make network request
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 403, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/pyt

ValueError: ignored

In [ ]:
!pip install tinygrad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.2/919.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
import requests

url = "http://tenant18.bot.5goil.tmobile.com/api/jobs"

payload = {
    "model": {
        "identifier": "dnaxchyxga",
        "version": "0.0.1"
    },
    "input": {
        "type": "embedded",
        "sources": { "user-defined-input-name": { "model-input-item-name": "..." } }
    },
    "explain": "false"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": "wZr4nkIAtLFPNUPySOdC.DYqB3325zWPXIDxdrZIH"
}

response = requests.post(url, json=payload, headers=headers, verify=False)

print(response.text)

ConnectionError: ignored